### Plotting yes or no (computationally expensive):

In [ ]:
plottting = False
plotting_testing = True

# References:

*   https://towardsdatascience.com/mapping-geograph-data-in-python-610a963d2d7f
*   https://developers.arcgis.com/python/samples/openstreetmap-exploration/



# Connect to drive folder

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# Install libraries

In [ ]:
%%capture
!pip install osmnx polars pyshp geofeather
#!pip uninstall largestinteriorrectangle
!pip install largestinteriorrectangle
#pyshp --> Python Shapefile Library (PyShp)

In [ ]:
# Data management
import pandas as pd
import datetime
import numpy as np
import polars as pl
import zipfile, io
import os
import numpy as np
import re
import functools
import urllib
import shutil
import warnings
from collections import Counter
import progressbar
from time import sleep
warnings.simplefilter("ignore")



# Geospatial libraries
import osmnx as ox
import shapefile as shp
import geopandas as gpd
import geofeather
import pyproj
import cv2 as cv # --> per largestinteriorrectangle

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# Importing the data

In [ ]:
def find_project_path(fold):
  for fold, subfold, file in os.walk(fold):      # os.walk() returns a generator object that yields a 3-tuple (dirpath, dirnames, filenames) for each directory in the directory tree
    if fold.endswith('Lab Smart Cities'):     #"Project Lab Smart Cities" (Tommaso)
      return fold

project_path = find_project_path(os.getcwd())   #Dovrebbe restituire
project_path

'/content/gdrive/MyDrive/Project Lab Smart Cities'

In [ ]:
datasets = {}

def get_geofeather_Files(subFolder_name):
  focus = subFolder_name
  if os.getcwd() != os.path.join(project_path, f'DATI/COMPLETE/{focus}/Geofeather'):
    os.chdir(os.path.join(project_path, f'DATI/COMPLETE/{focus}/Geofeather'))

  datasets[focus] = []

  for ext in os.listdir(os.getcwd()):
    try:
      datasets[focus].append(geofeather.from_geofeather(ext))   #get all files in the folder Geofeather, in order to print all of them
    except:
      continue
  return datasets[focus]   #return a list of file open in geofeather format


#DBT Milan (Shape of different elements, for official Milan Urban Planning)

In [ ]:
milano_shape_dbt = gpd.GeoDataFrame(get_geofeather_Files("MILANO")[0])
milano_shape_dbt = milano_shape_dbt.to_crs("EPSG:4326")
if plottting:
  milano_shape_dbt

In [ ]:
milano_shape_dbt[["codice_strato_tema", "classe" ]] = milano_shape_dbt["codice_classe"].apply(lambda x : pd.Series(str(x).split("_", maxsplit=1)))

**INTERESTING CODES**:

* AATT = 020206 Area attrezzata del suolo

* AC_CIC = 040101 Area bagnata di corso d'acqua

* ARGINE = 020502 Argine

* AR_VRD = 060401 Area verde

* A_TRAS = 050304 = Area in trasformazione o non strutturata

* CL_AGR = 060106 Coltura agricola

* EDIFC = 020102 Edificio

* INVASO = 040103 Invaso artificiale

* MU_DIV = 020210 Muro o divisione in spessore

* PS_INC = 060105 Pascolo o incolto

* SP_ACQ = 040102 Specchio d'acqua




**NOT INTERESTING**:


* AC_CIC = 010103 Area di circolazione ciclabile

* AC_PED = 010102 Area di circolazione pedonale

* AC_VEI = 010101 Area di circolazione veicolare

* AR_VMS = 010105 Viabilita' mista secondaria

* ATTR_SP = 020204 Attrezzatura sportiva

* BOSCO = 060101 Bosco

* EDI_MIN = 020106 Edificio minore

* FOR_PC = 060102 Formazione particolare

* F_NTER = 050301 Forma naturale del terreno

* MAN_TR = 020205 Manufatto d' infrastruttura di trasporto

* MN_IND = 020201 Manufatto industriale

* MN_MAU = 020202 Manufatto monumentale e di arredo urbano

* MU_SOS = 020401 Muro di sostegno e ritenuta del terreno

* OP_POR = 020505 Opera portuale e di difesa delle coste

* OP_REG = 020503 Opera idraulica di regolazione

* PONTE = 020301 Ponte/viadotto/cavalcavia

* SC_DIS = 050303 Area di scavo o discarica

* SD_FER = 010201 Sede di trasporto su ferro

* TRALIC = 020207 Sostegno a traliccio


# Explode milan dbt only with interesting codes

In [ ]:
milano_shape_dbt = milano_shape_dbt.astype({'codice_strato_tema':'int'})
classi_attributi_ok = ["AATT", "AC_CIC", "ARGINE", "AR_VRD", "A_TRAS", "CL_AGR" "EDIFC", "INVASO", "MU_DIV", "PS_INC", "SP_ACQ"]
df_leggero_exploded = milano_shape_dbt
if plottting:
  df_leggero_exploded

In [ ]:
df_leggero_exploded = gpd.GeoDataFrame(df_leggero_exploded).explode(index_parts=False)
if plottting:
  df_leggero_exploded

In [ ]:
df_leggero_exploded.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# Find Forsaken buildings area and point (from ImmDegrado file)

In [ ]:
immDegrado_point = get_geofeather_Files("ImmDegrado")[0]
immDegrado_area = get_geofeather_Files("ImmDegrado")[1]

immDegrado_area = immDegrado_area.to_crs("EPSG:4326")
immDegrado_point = immDegrado_point.to_crs("EPSG:4326") #.to_crs("EPSG:4326")

immDegrado_area["areaIndicator"] = gpd.GeoSeries(immDegrado_point["geometry"])
immDegrado_area["INDIRIZZO"] = immDegrado_area["INDIRIZZO"].astype(str)
immDegrado_area["Cod_prog"] = immDegrado_area["Cod_prog"].astype(str)
immDegrado_area["TIPO_MACRO"] = immDegrado_area["TIPO_MACRO"].astype(str)
immDegrado_area.drop(labels = ["LinkPdfVIG", "LinkPdfVAR"], axis = 1, inplace = True)

immDegrado_point['lon'] = immDegrado_point.geometry.apply(lambda p: p.x)
immDegrado_point['lat'] = immDegrado_point.geometry.apply(lambda p: p.y)

if plottting:
  display(immDegrado_point)

immDegrado_area.crs
immDegrado_point.crs


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# GigaChad function()
## Parameters:

* DBT
* Decaying area
* Decaying point



1. Neighbourhoods boundary
2. Streets boundary


* To compute:
    * Area with new buildings construction (new building)
    * Extended area with new buildings construction (new area new building)
    * Renovating buildings (renovation)

Check parameters:
* Isohypse control (boolean): to avoid the initial consideration of areas that are not level (generally those that were not man-made)
* Green control (boolean): to avoid cementing green areas even if they were disused/abandoned
* Sector_importance (boolean): give importance initially to commercial/manufacturing areas over residential areas
* Adjacency (boolean): create a unique area if the abandoned buildings are located on contiguous land (exception if rebuild area was set to false). If the area found is not more than 50/60% of the total area, find an additional 2nd or 3rd area for the building
* rebuild_area (boolean)

## Return

* Dataset with new information and area classifications






In [ ]:
#It will be used for 1 scenario:
# - NEW AREA AND NEW BUILING

from shapely.ops import cascaded_union
from shapely.ops import unary_union
from shapely.ops import Polygon

#For Debugging
'''
def findAdjacentAreas (edificiAbbandonati):

  #Buffer of some meters to avoid intersenction problem due to inprecisions
  edificiAbbandonati['geometry_buffered'] = edificiAbbandonati.geometry.buffer(0.00001, join_style = 2)
  edificiAbbandonati = edificiAbbandonati.reset_index()

  #Avoid some areas which where for a Residential destination usage
  avoidSomeClasses = ["RESIDENZIALE"]  #
  avoidUnion = edificiAbbandonati[edificiAbbandonati["TIPO_MACRO"].isin(avoidSomeClasses)].index.values.tolist()
  print(avoidUnion)

  #listing all points which are already joined togheter and placed in "new_polygons"
  alreadyJoined = []
  new_polygons = []
  print(f'Type df {type(edificiAbbandonati.geometry_buffered)}')
  for index, r in edificiAbbandonati.iterrows():
    print(f'Type {type(r.geometry_buffered)}')
    if (index not in avoidUnion) and (index not in alreadyJoined):
      neighbors = edificiAbbandonati[ ~ edificiAbbandonati.geometry_buffered.disjoint(r.geometry_buffered)].index.tolist()    #tilde means "not"
      neighbors= list(set(neighbors).difference(avoidUnion))   #disjoin take in consideration also elements which were in the "avoidUnion" list--> must be removed
      neighbors.remove(index)  #disjoin take in consideration also the index itself --> must be removed

      print("--------------------------------------")
      print(index)
      print(f'first cycle, {neighbors}')

      for index_Neigh in neighbors:

        second_neighbors = edificiAbbandonati[ ~ edificiAbbandonati.geometry_buffered.disjoint(edificiAbbandonati.geometry_buffered[index_Neigh])].index.tolist()
        neighbors.extend(x for x in second_neighbors if ((x not in neighbors) and (x != index) and (x not in avoidUnion ) and (x not in alreadyJoined)))   #It should extend it at runtime!!
        print(f'second cycle, {neighbors}')

      alreadyJoined.append(index)
      alreadyJoined.extend(neighbors)

      print(f'found these new FINALS neighbors, {neighbors}')

      #Union
      polygons = [edificiAbbandonati.geometry_buffered[i] for i in neighbors]
      polygons.append(edificiAbbandonati.geometry_buffered[index])
      print( f' len polygons , {len(polygons)}')

      new_polygons.append(unary_union(polygons).buffer(-0.00001, join_style = 2))    #unary_union     #dissolve
      print( f' len new_polygons , {len(new_polygons)}')
      print(f'NON PIU\' CONSIDERATI , {alreadyJoined}')

    elif (index in avoidUnion):
      new_polygons.append(edificiAbbandonati.geometry[index])
  return new_polygons
  '''


def findAdjacentAreas (edificiAbbandonati):

  #Buffer of some meters to avoid intersenction problem due to inprecisions
  edificiAbbandonati
  edificiAbbandonati['geometry_buffered'] = edificiAbbandonati.geometry.buffer(0.00001, join_style = 2)
  edificiAbbandonati = edificiAbbandonati.reset_index()

  #Avoid some areas which where for a Residential destination usage
  avoidSomeClasses = []  #"RESIDENZIALE"
  avoidUnion = edificiAbbandonati[edificiAbbandonati["TIPO_MACRO"].isin(avoidSomeClasses)].index.values.tolist()

  #listing all points which are already joined togheter and placed in "new_polygons"
  alreadyJoined = []
  new_polygons = []

  for index, r in edificiAbbandonati.iterrows():
    if (index not in avoidUnion) and (index not in alreadyJoined):
      neighbors = edificiAbbandonati[ ~ edificiAbbandonati.geometry_buffered.disjoint(r.geometry_buffered)].index.tolist()    #tilde means "not"
      neighbors= list(set(neighbors).difference(avoidUnion))   #disjoin take in consideration also elements which were in the "avoidUnion" list--> must be removed
      neighbors.remove(index)  #disjoin take in consideration also the index itself --> must be removed

      for index_Neigh in neighbors:

        second_neighbors = edificiAbbandonati[ ~ edificiAbbandonati.geometry_buffered.disjoint(edificiAbbandonati.geometry_buffered[index_Neigh])].index.tolist()
        neighbors.extend(x for x in second_neighbors if ((x not in neighbors) and (x != index) and (x not in avoidUnion ) and (x not in alreadyJoined)))   #It should extend it at runtime!!

      alreadyJoined.append(index)
      alreadyJoined.extend(neighbors)

      #Union
      polygons = [edificiAbbandonati.geometry_buffered[i] for i in neighbors]
      polygons.append(edificiAbbandonati.geometry_buffered[index])

      new_polygons.append(unary_union(polygons).buffer(-0.00001, join_style = 2))    #unary_union     #dissolve

    elif (index in avoidUnion):
      new_polygons.append(edificiAbbandonati.geometry[index])
  d = {"geometry": new_polygons}
  return gpd.GeoDataFrame(d)

In [ ]:
edifici_uniti = gpd.GeoDataFrame(findAdjacentAreas(immDegrado_area))
edifici_uniti

,geometry
0,POLYGON EMPTY
1,POLYGON EMPTY
2,POLYGON EMPTY
3,POLYGON EMPTY
4,POLYGON EMPTY
...,...
173,POLYGON EMPTY
174,POLYGON EMPTY
175,POLYGON EMPTY
176,POLYGON EMPTY


In [ ]:
from pyproj import Geod
from shapely.geometry import LineString, Point, Polygon
count = 0
for i, r in immDegrado_area.iterrows():
  geod = Geod(ellps="WGS84")
  poly_area, poly_perimeter = geod.geometry_area_perimeter(immDegrado_area["geometry"][i])
  count = count + poly_area
count/1000000

0.0

In [ ]:
from shapely.geometry import Polygon, MultiPolygon
#It will be used for 1 cenario:
# - BUILDING RENOVATION
def buildingRenovation (edificiAbbandonati, DBT, largestORtwo = True):

  '''
  If The builiding is outside for at least the 35% of the surface we will consider it as an external un-considerd building
  '''

  if largestORtwo == True:

    DBT = gpd.GeoDataFrame(DBT).explode(index_parts=False)
    edificiAbbandonati["buildings_geometry"] = ""
    edificiAbbandonati["area"] = ""
    edificiAbbandonati["volume"] = ""
    edificiAbbandonati = edificiAbbandonati.reset_index()
    edificiAbbandonati["original_index"] = edificiAbbandonati.index
    DBT = DBT.reset_index()
    DBT = DBT[DBT["classe"] == "EDIFC"]

    for index, r in edificiAbbandonati.iterrows():

      internalBuilidings = DBT[(DBT.geometry.intersects(r.geometry))]
      internalBuilidings["area"] = [0] * len(internalBuilidings)
      for indexInternal, rInternal in internalBuilidings.iterrows():
        geod = Geod(ellps="WGS84")
        poly_area, poly_perimeter = geod.geometry_area_perimeter(rInternal["geometry"])
        internalBuilidings["area"][indexInternal] = poly_area


      #More inside
      if len(internalBuilidings) > 1:
        for index_find, r_find in internalBuilidings.iterrows():
          if (r_find["geometry"].difference(r["geometry"]).area >= 0.35 * r_find["geometry"].area):
            internalBuilidings.drop(labels = index_find, axis = 0, inplace = True)

        #MORE than one remaining after drop
        if len(internalBuilidings) > 1:
          if sorted((internalBuilidings["area"]).tolist())[-1] * 0.50 > sorted(internalBuilidings["area"].tolist())[-2]:
            MultiPoli = []
            MultiArea = 0
            for i_a , r_a  in internalBuilidings.iterrows():
              MultiPoli.append(r_a["geometry"])
              MultiArea = MultiArea + r_a["area"]
            edificiAbbandonati["buildings_geometry"][index] = MultiPoli
            edificiAbbandonati["area"][index] = MultiArea
            edificiAbbandonati["volume"][index] = edificiAbbandonati["area"][index] * 10
          else:
            edificio = internalBuilidings[internalBuilidings["area"] == (sorted(internalBuilidings["area"].tolist())[-1])]
            edificiAbbandonati["buildings_geometry"][index] = [edificio["geometry"].tolist()[0]]
            edificiAbbandonati["area"][index] = edificio["area"].tolist()[0]
            edificiAbbandonati["volume"][index] = edificiAbbandonati["area"][index] * 10

        #One remaining after drop
        if len(internalBuilidings) == 1:
          for index_find, r_find in internalBuilidings.iterrows():
            edificiAbbandonati["buildings_geometry"][index] = [r_find["geometry"]]
            edificiAbbandonati["area"][index] = r_find["area"]
            edificiAbbandonati["volume"][index] = edificiAbbandonati["area"][index]  * 10

        #All dropped
        if len(internalBuilidings) == 0:
          edificiAbbandonati.drop(labels = index, axis = 0, inplace = True)

      #Only one inside
      elif len(internalBuilidings) == 1:
        for index_find, r_find in internalBuilidings.iterrows():
          edificiAbbandonati["buildings_geometry"][index] = [r_find["geometry"]]
          edificiAbbandonati["area"][index] =  r_find["area"]
          edificiAbbandonati["volume"][index] = edificiAbbandonati["area"][index]  * 10

      #NO building inside
      elif len(internalBuilidings) == 0:
        edificiAbbandonati.drop(labels = index, axis = 0, inplace = True)
  edificiAbbandonati = edificiAbbandonati.reset_index()
  return gpd.GeoDataFrame(edificiAbbandonati)

In [ ]:
buildingRenovationDt = buildingRenovation(immDegrado_area, df_leggero_exploded, largestORtwo = True)
buildingRenovationDt.to_csv(project_path + '/DATI/FINAL/renovation/renovation_complete.csv')

In [ ]:
renovation_areaGeom = buildingRenovation["geometry"]
renovation_areaGeom.to_file(project_path + '/DATI/FINAL/renovation/renovation_areaGeom.gpkg', driver='GPKG')

In [ ]:
#It will be used for 2 different scenarios:
# - NEW BUILDING,
# - NEW AREA AND NEW BUILING

from shapely.geometry.base import CAP_STYLE
from shapely.geometry import Polygon, MultiPolygon
from shapely import wkt
from shapely.wkt import loads

#CHECK IF THERE IS A DIFFERENCE WITH ITALIAN SYSTEM GEOCODE EPSG:4326!!!
#https://en.wikipedia.org/wiki/Decimal_degrees#:~:text=Precision,-The%20radius%20of&text=A%20value%20in%20decimal%20degrees,8%20in)%20at%20the%20equator.

#buffer edifici limitrofi,
#checking se vuoe area unificata--> devo unire aree passate in input. Unisco solo quelli che non sono con edifici relativi a immobili abitativi
def define_newArea_building(edificiAbbandonati, DBT, IF_biggerAreaConstruction = False, distEdific = 0 , distFinestrato = 0, distPerimetroFondi = 0 ):
  '''
  Funzione che, prende tutte le aree, ne fa il buffer, considera se devono essere distanti da: strade(e di quanto), edifici residenziali(e diquanto) altri edifici (e di quanto)
  Prende il massimo di questi valori e fa il buffer per quei metri.
  Se interseca qualcosa mi facci restituire l'indice dell'edificio/strada etc.
  Ne calcolo il buffer di questi a norma di legge di distanze
  Taglio l'area dell'edificio abbandonato
  Restituisce di fatto l'area di massima estenzione di un nuovo edificio
  '''

  distEdific = distEdific/(1.11 * 100000)
  distFinestrato = distFinestrato/(1.11 * 100000)
  distPerimetroFondi = distPerimetroFondi/(1.11 * 100000)

  dataset_areePotenziali_edifici = None
  df_aree_degrado = None
  max_distance = max([distEdific, distFinestrato])

  to_delete = []

  if IF_biggerAreaConstruction == True:
    df_aree_degrado = gpd.GeoDataFrame(findAdjacentAreas(edificiAbbandonati))
    df_aree_degrado = gpd.GeoDataFrame(df_aree_degrado).explode(index_parts=False)
    df_aree_degrado = df_aree_degrado.reset_index()
  else:
    df_aree_degrado = edificiAbbandonati
    #Explode for any Multipolygons already in it
    df_aree_degrado = gpd.GeoDataFrame(df_aree_degrado).explode(index_parts=False)
    df_aree_degrado = df_aree_degrado.reset_index()



  df_aree_degrado["geometry_newArea"] = df_aree_degrado["geometry"].buffer(-distPerimetroFondi, join_style = 2)

  '''
  A quanto pare non gli piacciono i Multypolygon con il within
  listPolygonsDegrado = df_aree_degrado.geometry.to_wkt().tolist()
  all_pgons = [shapely.wkt.loads(pgon) for pgon in listPolygonsDegrado]

  # Create the required multipolygon
  multi_pgon = MultiPolygon(all_pgons)
  '''

  df_aree_degrado['area_buffered'] = df_aree_degrado.geometry.buffer(max_distance, join_style = 2) #, join_style = 2        ,join_style = 3, cap_style = 3
  for index, r in df_aree_degrado.iterrows():
    print(f"index:", {index},"/",{len(df_aree_degrado)})

    #Find all building if the boundary or interior of the object intersect in any way with those of the buffered area.
    # AND are not complitly covered by the area of Degrado Imm (not buffered) --> Not possible to use covered_by cause the rpesence of building n
    #Checking the intersection at 80%
    list_partialOutBoundery = []
    DBT['old_index'] = DBT.index
    Inters  = DBT[DBT.geometry.intersects(r.geometry)]
    for index_Inters, r_Inters in Inters.iterrows():
      if (r_Inters["geometry"].difference(r.geometry).area <= 0.50 * r_Inters["geometry"].area ):
        list_partialOutBoundery.append(r_Inters.old_index)
    df_zone_EDIFIC_limitrofe = DBT[(DBT.geometry.intersects(r.area_buffered))  & ( ~ DBT.geometry.buffer(- 1/(1.11 * 100000)).covered_by(r.geometry)) & ( ~ DBT.index.isin(list_partialOutBoundery)) ]     #


    #Create new area building
    for index_LIM, r_LIM in df_zone_EDIFIC_limitrofe.iterrows():
      if (r_LIM.classe == "EDIFC") & (distEdific != 0):
        df_aree_degrado["geometry_newArea"][index] = df_aree_degrado["geometry_newArea"][index].difference(r_LIM.geometry.buffer(distEdific, join_style = 2 ))
      if (r_LIM.classe == "EDIFC") & (distFinestrato != 0):
        df_aree_degrado["geometry_newArea"][index] = df_aree_degrado["geometry_newArea"][index].difference(r_LIM.geometry.buffer(distFinestrato, join_style = 2 ))


  '''
  for index, r in DBT.iterrows():
    #rimuovere edifici gia inseriti dentro queste aree
    #Fare buffer delle aree
    print(countdown)
    countdown -= 1
    for index_A, r_A in df_aree_degrado.iterrows():
      if r.geometry.within(r_A.geometry):
        to_delete.append(index)
  '''

  return gpd.GeoDataFrame(df_aree_degrado) #to_delete



In [ ]:
df_aree_degrado_SameArea = define_newArea_building(immDegrado_area, df_leggero_exploded, distEdific = 10, distFinestrato = 5, distPerimetroFondi = 5, IF_biggerAreaConstruction = False)

index: {0} / {184}
index: {1} / {184}
index: {2} / {184}
index: {3} / {184}
index: {4} / {184}
index: {5} / {184}
index: {6} / {184}
index: {7} / {184}
index: {8} / {184}
index: {9} / {184}
index: {10} / {184}
index: {11} / {184}
index: {12} / {184}
index: {13} / {184}
index: {14} / {184}
index: {15} / {184}
index: {16} / {184}
index: {17} / {184}
index: {18} / {184}
index: {19} / {184}
index: {20} / {184}
index: {21} / {184}
index: {22} / {184}
index: {23} / {184}
index: {24} / {184}
index: {25} / {184}
index: {26} / {184}
index: {27} / {184}
index: {28} / {184}
index: {29} / {184}
index: {30} / {184}
index: {31} / {184}
index: {32} / {184}
index: {33} / {184}
index: {34} / {184}
index: {35} / {184}
index: {36} / {184}
index: {37} / {184}
index: {38} / {184}
index: {39} / {184}
index: {40} / {184}
index: {41} / {184}
index: {42} / {184}
index: {43} / {184}
index: {44} / {184}
index: {45} / {184}
index: {46} / {184}
index: {47} / {184}
index: {48} / {184}
index: {49} / {184}
index: {50

In [ ]:
df_aree_degrado_SameArea.to_csv(project_path + '/DATI/FINAL/new_building/new_buildingNORECT.csv')

In [ ]:
df_aree_degrado_BiggerArea = define_newArea_building(immDegrado_area, df_leggero_exploded, distEdific = 10, distFinestrato = 10, distPerimetroFondi = 5, IF_biggerAreaConstruction = True)

index: {0} / {169}
index: {1} / {169}
index: {2} / {169}
index: {3} / {169}
index: {4} / {169}
index: {5} / {169}
index: {6} / {169}
index: {7} / {169}
index: {8} / {169}
index: {9} / {169}
index: {10} / {169}
index: {11} / {169}
index: {12} / {169}
index: {13} / {169}
index: {14} / {169}
index: {15} / {169}
index: {16} / {169}
index: {17} / {169}
index: {18} / {169}
index: {19} / {169}
index: {20} / {169}
index: {21} / {169}
index: {22} / {169}
index: {23} / {169}
index: {24} / {169}
index: {25} / {169}
index: {26} / {169}
index: {27} / {169}
index: {28} / {169}
index: {29} / {169}
index: {30} / {169}
index: {31} / {169}
index: {32} / {169}
index: {33} / {169}
index: {34} / {169}
index: {35} / {169}
index: {36} / {169}
index: {37} / {169}
index: {38} / {169}
index: {39} / {169}
index: {40} / {169}
index: {41} / {169}
index: {42} / {169}
index: {43} / {169}
index: {44} / {169}
index: {45} / {169}
index: {46} / {169}
index: {47} / {169}
index: {48} / {169}
index: {49} / {169}
index: {50

In [ ]:
df_aree_degrado_BiggerArea.to_csv(project_path + '/DATI/FINAL/new_area_new_building/new_area_new_buildingNORECT.csv')

In [ ]:
#It will be used for 2 different scenarios:
# - NEW BUILDING,
# - NEW AREA AND NEW BUILING
import math
def create_hexagon(l, x, y):
    """
    Create a hexagon centered on (x, y)
    :param l: length of the hexagon's edge
    :param x: x-coordinate of the hexagon's center
    :param y: y-coordinate of the hexagon's center
    :return: The polygon containing the hexagon's coordinates
    """
    c = [[x + math.cos(math.radians(angle)) * l, y + math.sin(math.radians(angle)) * l] for angle in range(0, 360, 60)]
    return Polygon(c)

In [ ]:
#It will be used for 2 different scenarios:
# - NEW BUILDING,
# - NEW AREA AND NEW BUILING
def create_hexgrid(bbox, side):
    """
    returns an array of Points describing hexagons centers that are inside the given bounding_box
    :param bbox: The containing bounding box. The bbox coordinate should be in Webmercator.
    :param side: The size of the hexagons'
    :return: The hexagon grid
    """
    grid = []

    v_step = math.sqrt(3) * side
    h_step = 1.5 * side

    x_min = min(bbox[0], bbox[2])
    x_max = max(bbox[0], bbox[2])
    y_min = min(bbox[1], bbox[3])
    y_max = max(bbox[1], bbox[3])

    h_skip = math.ceil(x_min / h_step) - 1
    h_start = h_skip * h_step

    v_skip = math.ceil(y_min / v_step) - 1
    v_start = v_skip * v_step

    h_end = x_max + h_step
    v_end = y_max + v_step

    if v_start - (v_step / 2.0) < y_min:
        v_start_array = [v_start + (v_step / 2.0), v_start]
    else:
        v_start_array = [v_start - (v_step / 2.0), v_start]

    v_start_idx = int(abs(h_skip) % 2)

    c_x = h_start
    c_y = v_start_array[v_start_idx]
    v_start_idx = (v_start_idx + 1) % 2
    while c_x < h_end:
        while c_y < v_end:
            grid.append((c_x, c_y))
            c_y += v_step
        c_x += h_step
        c_y = v_start_array[v_start_idx]
        v_start_idx = (v_start_idx + 1) % 2

    return grid

In [ ]:
df_aree_degrado_SameArea = pd.read_csv(project_path + '/DATI/FINAL/new_buildingRECTA.csv')
df_aree_degrado_SameArea = gpd.GeoDataFrame(df_aree_degrado_SameArea)
df_aree_degrado_SameArea["final_geometry_rectangular"] = gpd.GeoSeries.from_wkt(df_aree_degrado_SameArea["final_geometry_rectangular"])
df_aree_degrado_SameArea["geometry_newArea"] = gpd.GeoSeries.from_wkt(df_aree_degrado_SameArea["geometry_newArea"])

NameError: ignored

In [ ]:
edge = 3/(1.11 * 100000)
df_aree_degradocdscscds = df_aree_degrado_SameArea.reset_index()
hex_centers = create_hexgrid(df_aree_degradocdscscds.geometry_newArea[109].bounds, edge)

In [ ]:
if True:
  hex_centers = gpd.GeoDataFrame(hex_centers, columns = ["lon","lat"])
  hex_centers["geometry_point_approximated"] = gpd.points_from_xy(hex_centers.lon, hex_centers.lat, crs="EPSG:4326" ) #NOTO CHE LE GEOMETRY SONO APPROSSIMATE RISPETTO AI PUNTI REALI CREATI!!!
  hexagons_poly = [create_hexagon(3/(1.11 * 100000), r.lon, r.lat) for i , r in hex_centers.iterrows()] # [edificiAbbandonati.geometry_buffered[i] for i in neighbors]

  hex_centers["geometry_hexagon"] = hexagons_poly
  hex_centers["geometry_hexagon"] = gpd.GeoSeries(hex_centers["geometry_hexagon"])
  hex_centers["geometry_hexagon"] = hex_centers["geometry_hexagon"].buffer(0.5/(1.11 * 100000))
  hex_centers = hex_centers[hex_centers.geometry_hexagon.within(df_aree_degrado_SameArea.geometry_newArea[109])]
  hexagons_poly =  gpd.GeoSeries(hexagons_poly)
  hexagons_poly = unary_union(hexagons_poly)

  type(hexagons_poly)

In [ ]:
#It will be used for 2 different scenarios:
# - NEW BUILDING,
# - NEW AREA AND NEW BUILING
''' FOR DEBUGGING
from shapely.ops import triangulate
from shapely.geometry import Polygon, LinearRing

def find_Largest_Rectangular (aree_EdificiAbbandonati):

  aree_EdificiAbbandonati = aree_EdificiAbbandonati.reset_index()

  aree_EdificiAbbandonati["final_geometry_convex"] = ""
  aree_EdificiAbbandonati["final_area_convex"] = ""
  aree_EdificiAbbandonati["final_geometry_rectangular"] = ""
  aree_EdificiAbbandonati["final_area_rectangular"] = ""

  #For each area
  for index, r in aree_EdificiAbbandonati.iterrows():

    print(f'\n-----------------------------------------\n',
          f'\n-----------------------------------------\n',
          f'Index Degrado Area examinated: {index}')

    final_geometry_convex = None
    final_area_convex = 0

    edge = 3/(1.11 * 100000)
    hex_centers = create_hexgrid(r.geometry_newArea.bounds, edge)
    hex_centers = gpd.GeoDataFrame(hex_centers, columns = ["lon","lat"])
    hexagons_poly = [create_hexagon(3/(1.11 * 100000), r.lon, r.lat) for i , r in hex_centers.iterrows()] # [edificiAbbandonati.geometry_buffered[i] for i in neighbors]

    hex_centers["delaunay_triangulation"]                    = hexagons_poly
    aree_EdificiAbbandonati["delaunay_triangulation"][index] = hexagons_poly

    #FORSE ANCORA MEGLIO DI COVERED E' MEGLIO TAGLIARE I TRIANCOLI
    list_trinagles_within =  [trian for trian in aree_EdificiAbbandonati["delaunay_triangulation"][index] if (trian.covered_by(r.geometry_newArea))]

    #aree_EdificiAbbandonati["delaunay_triangulation"][index] = list_trinagles_within
    #Create the rectangular inside the largest convex subset of triangles existing

    df = hex_centers
    df = df.reset_index()
    df = gpd.GeoDataFrame(df)
    df["delaunay_triangulation"] = gpd.GeoSeries(df["delaunay_triangulation"])
    df = df[df.delaunay_triangulation.within(r.geometry_newArea)]
    df = df.reset_index()

    if (len(df) > 0) and (not(df["delaunay_triangulation"].isnull().values.any())):

      #For each hexagon
      for index_tri, r_tri in df.iterrows():
        print("\n\n\n\n-- FASE INIZIALE itezaione su ogni triangolo trovato:")
        print(f'index {index_tri}/{len(df)}\n')

        #Avoid hexagon which create a NON convex area
        avoidUnion = [index_tri]
        alreadyJoined = [index_tri]
        new_polygons = []

        neighbors_indexList = df[~ df.delaunay_triangulation.disjoint(r_tri.delaunay_triangulation)].index.tolist()    #tilde means "not"

        if index in neighbors_indexList:
          neighbors_indexList.remove(index)
        print(f"Neighbors_indexList INITIAL LIST of NEAREST HEXAGON: \n{neighbors_indexList}")
        print(f"Already joined: \n{alreadyJoined}")
        print(f"Avoid union: \n{avoidUnion}")

        neighbors_df = df[ ~ df.delaunay_triangulation.disjoint(r_tri.delaunay_triangulation)]
        neighbors_df.drop(index_tri)

        max_subset_triangles_geometry = r_tri.delaunay_triangulation
        max_subset_triangles_area     = r_tri.delaunay_triangulation.area

        for index_nei in neighbors_indexList:
          print("fase 1")
          print(f"VALUTANDO L'INDICE NUMERO: {index_nei}")
          if (unary_union([max_subset_triangles_geometry, neighbors_df.delaunay_triangulation[index_nei]])
                          .minimum_rotated_rectangle
                          .within(r.geometry_newArea) and       #FORSE BISOGNA CAMBIARE ANCHE L'equals IN QUALCOSA DI MENO AGGRESSIVO
                          (index_nei not in avoidUnion)):
            print("fase 2")
            union_triangles = unary_union([max_subset_triangles_geometry, neighbors_df.delaunay_triangulation[index_nei]])
            #if union_triangles.area > max_subset_triangles_area :
            print("fase 3")
            max_subset_triangles_geometry = union_triangles
            print("fase 3.1")
            max_subset_triangles_area = union_triangles.area
            print("fase 3.2")
            alreadyJoined.append(index_nei)
            print("fase 3.4\n")
            print(f"Neighbors_indexList ACTUAL: \n{neighbors_indexList}")
            print(f"Already joined: \n{alreadyJoined}")
            #print(f"Avoid union: \n{avoidUnion}")
            #SE E' PESANTE COMPUTAZIONALMENTE POTREI FARGLI ESTENDERE LA LISTA SOLO QUANDO NON RIMANE UN SOLO ULTIMO ELEMENTO NELLA LISTA  neighbors_indexList!!!!
            neighbors_indexList.extend([x for x in df[ ~ df.delaunay_triangulation.disjoint(max_subset_triangles_geometry)].index.tolist() if((x not in alreadyJoined) and (x not in avoidUnion) and (x not in neighbors_indexList))])
            neighbors_df = df[ ~ df.delaunay_triangulation.disjoint(max_subset_triangles_geometry)]
            print(f"Neighbors_indexList EXPANDED: \n{neighbors_indexList}")
            print("\nfase 3.5")
          else :
            print("fase 4")
            if index_nei not in avoidUnion:
              avoidUnion.append(index_nei)
              print(f"ADDED TO Avoid union: \n{index_nei}")
              #print(f"Avoid union: \n{avoidUnion}")

        if max_subset_triangles_area > final_area_convex :
          print(f"\n Neighboroud finished united: \n{alreadyJoined}")
          print("fase 10", "----------------->",  max_subset_triangles_geometry)
          final_geometry_convex = max_subset_triangles_geometry
          print("fase 11", "----------------->",  max_subset_triangles_area)
          final_area_convex = max_subset_triangles_area

    print("\n fase 20 =====================>", final_geometry_convex)
    aree_EdificiAbbandonati["final_geometry_convex"][index] = final_geometry_convex
    aree_EdificiAbbandonati["final_area_convex"][index] = final_area_convex
    aree_EdificiAbbandonati["final_geometry_rectangular"][index] = final_geometry_convex.minimum_rotated_rectangle
    aree_EdificiAbbandonati["final_area_rectangular"][index] = final_geometry_convex.minimum_rotated_rectangle.area

  return gpd.GeoDataFrame(aree_EdificiAbbandonati)
'''

import shapely.geometry
from shapely.ops import triangulate
from shapely.geometry import Polygon, LinearRing
from shapely.geometry import Point
import shapely.wkt


def find_Largest_Rectangular (aree_EdificiAbbandonati):

  aree_EdificiAbbandonati = aree_EdificiAbbandonati.reset_index()

  aree_EdificiAbbandonati["delaunay_triangulation"] = ""
  aree_EdificiAbbandonati["final_geometry_convex"] = ""
  aree_EdificiAbbandonati["final_area_convex"] = ""
  aree_EdificiAbbandonati["final_geometry_rectangular"] = ""
  aree_EdificiAbbandonati["final_area_rectangular"] = ""

  #For each area
  for index, r in aree_EdificiAbbandonati.iterrows():

    print(f'\n-----------------------------------------\n',
      f'\n-----------------------------------------\n',
      f'Index Degrado Area examinated: {index}',
      f'\n-----------------------------------------\n',
      f'\n-----------------------------------------\n', )

    final_geometry_convex = None
    final_area_convex = 0

    if  (not r.geometry_newArea.is_empty) or (r.geometry_newArea == None):


      edge = 3/(1.11 * 100000)
      hex_centers = create_hexgrid(r.geometry_newArea.bounds, edge)
      hex_centers = gpd.GeoDataFrame(hex_centers, columns = ["lon","lat"])
      hexagons_poly = [create_hexagon(3/(1.11 * 100000), r.lon, r.lat) for i , r in hex_centers.iterrows()]

      aree_EdificiAbbandonati["delaunay_triangulation"][index] = hexagons_poly
      hex_centers["delaunay_triangulation"] = hexagons_poly
      hex_centers["delaunay_triangulation"] = gpd.GeoSeries(hex_centers["delaunay_triangulation"])
      hex_centers["delaunay_triangulation"] = hex_centers["delaunay_triangulation"].buffer(0.1/(1.11 * 100000))


      list_trinagles_within =  [trian for trian in aree_EdificiAbbandonati["delaunay_triangulation"][index] if (trian.covered_by(r.geometry_newArea))]

      #Create the rectangular inside the largest convex subset of triangles existing
      df = hex_centers
      df = df.reset_index()
      df = gpd.GeoDataFrame(df)
      df["delaunay_triangulation"] = gpd.GeoSeries(df["delaunay_triangulation"])
      df = df[df.delaunay_triangulation.within(r.geometry_newArea)]
      df = df.reset_index()

      if (len(df) > 0) and (not(df["delaunay_triangulation"].isnull().values.any())):

        count_maximal_reached = -1
        #For each hexagon
        for index_tri, r_tri in df.iterrows():

          count_maximal_reached +=1

          #Avoid hexagon which create a NON convex area
          avoidUnion = [index_tri]
          alreadyJoined = [index_tri]
          new_polygons = []

          neighbors_indexList = df[~ df.delaunay_triangulation.disjoint(r_tri.delaunay_triangulation)].index.tolist()    #tilde means "not"

          if index in neighbors_indexList:
            neighbors_indexList.remove(index)

          neighbors_df = df[ ~ df.delaunay_triangulation.disjoint(r_tri.delaunay_triangulation)]
          neighbors_df.drop(index_tri)

          max_subset_triangles_geometry = r_tri.delaunay_triangulation
          max_subset_triangles_area     = r_tri.delaunay_triangulation.area

          for index_nei in neighbors_indexList:

            union_triangles = unary_union([max_subset_triangles_geometry, neighbors_df.delaunay_triangulation[index_nei]])
            if (union_triangles.minimum_rotated_rectangle
                            .within(r.geometry_newArea) and
                            (index_nei not in avoidUnion)):

              max_subset_triangles_geometry = union_triangles
              max_subset_triangles_area = union_triangles.area
              alreadyJoined.append(index_nei)

              neighbors_indexList.extend([x for x in df[ ~ df.delaunay_triangulation.disjoint(max_subset_triangles_geometry)].index.tolist() if((x not in alreadyJoined) and (x not in avoidUnion) and (x not in neighbors_indexList))])
              neighbors_df = df[ ~ df.delaunay_triangulation.disjoint(max_subset_triangles_geometry)]

            else :
              if index_nei not in avoidUnion:
                avoidUnion.append(index_nei)

          if max_subset_triangles_area > final_area_convex :
            final_geometry_convex = max_subset_triangles_geometry
            final_area_convex = max_subset_triangles_area
            count_maximal_reached = 0


          #STOP CRITERIA:
          if (count_maximal_reached >= 0.30 * len(df) ) and ((final_area_convex >= 0.80 * r.geometry_newArea.area) or ((index_tri/len(df) >= 0.60) and (len(df) > 500) )):
            break


      print("\n Final Geometry =====================>", final_geometry_convex)
      aree_EdificiAbbandonati["final_geometry_convex"][index] = final_geometry_convex
      aree_EdificiAbbandonati["final_area_convex"][index] = final_area_convex
      if final_geometry_convex != None:
        aree_EdificiAbbandonati["final_geometry_rectangular"][index] = final_geometry_convex.minimum_rotated_rectangle
        aree_EdificiAbbandonati["final_area_rectangular"][index] = final_geometry_convex.minimum_rotated_rectangle.area
      else:
        aree_EdificiAbbandonati.drop(labels = index, axis = 0, inplace = True)
    else:
      aree_EdificiAbbandonati.drop(labels = index, axis = 0, inplace = True)

  aree_EdificiAbbandonati["final_geometry_convex"] = gpd.GeoSeries(aree_EdificiAbbandonati["final_geometry_convex"])
  aree_EdificiAbbandonati["final_geometry_rectangular"] = gpd.GeoSeries(aree_EdificiAbbandonati["final_geometry_rectangular"])
  aree_EdificiAbbandonati["final_area_rectangular"] = aree_EdificiAbbandonati["final_area_rectangular"].astype(float)
  aree_EdificiAbbandonati["final_area_convex"] = aree_EdificiAbbandonati["final_area_convex"].astype(float)
  aree_EdificiAbbandonati["delaunay_triangulation"] = gpd.GeoSeries([MultiPolygon(aree_EdificiAbbandonati["delaunay_triangulation"][i]) for i in range(len(aree_EdificiAbbandonati))])

  return gpd.GeoDataFrame(aree_EdificiAbbandonati)



In [ ]:
#aree_EdificiAbbandonati["delaunay_triangulation"] = aree_EdificiAbbandonati["geometry"]

range_remainingList = [*range(len(df_aree_degrado) -10, len(df_aree_degrado))]
aa = find_Largest_Rectangular(df_aree_degrado[df_aree_degrado.index.isin(range_remainingList)])



-----------------------------------------
 
-----------------------------------------
 Index Degrado Area examinated: 0 
-----------------------------------------
 
-----------------------------------------


 Final Geometry =====================> POLYGON ((9.185568124467554 45.456432213613546, 9.185568054631366 45.45643216388343, 9.185567980384253 45.45643212101683, 9.185567902398608 45.456432085401985, 9.185567821380682 45.456432057361404, 9.185567738064183 45.45643203714904, 9.185567653203641 45.45643202494794, 9.185567567567567 45.45643202086859, 9.18554054054054 45.45643202086859, 9.185540454904466 45.45643202494794, 9.185540370043924 45.45643203714904, 9.185540286727425 45.456432057361404, 9.185540205709499 45.456432085401985, 9.185540127723854 45.45643212101683, 9.18554005347674 45.45643216388343, 9.185539983640552 45.456432213613546, 9.185539918847736 45.456432269756846, 9.185539859685067 45.45643233180486, 9.18553980668833 45.4564323991957, 9.185539760337473 45.45643247131904

In [ ]:
#It will be used for all 3 different scenarios:
# - BUILDING RENOVATION,
# - NEW BUILDING,
# - NEW AREA AND NEW BUILING

def estimate_TotalProduction (aree_EdificiAbbandonati_and_rectangularArea, DBT):
  print()
  '''
  This function will estimate the total production of the vertical hub for all 3 different production methods
  '''

In [ ]:
#It will be used for all 3 different scenarios:
# - BUILDING RENOVATION,
# - NEW BUILDING,
# - NEW AREA AND NEW BUILING

def find_CityAreaCovered_FootAndBike_OnTotalProduction (aree_EdificiAbbandonati_and_rectangularArea_and_ProductionScenarios, DBT):
  '''
  This function take the
  '''

In [ ]:
aa_othersElements = aa[["index", "Cod_prog", "INDIRIZZO",  "TIPO_MACRO", "Shape_Area", "areaIndicator", "final_area_rectangular"]]

In [ ]:
aa.dtypes

index                            int64
Cod_prog                        object
INDIRIZZO                       object
TIPO_MACRO                      object
Shape_Area                     float64
areaIndicator                 geometry
geometry_buffered             geometry
geometry                      geometry
geometry_newArea              geometry
area_buffered                 geometry
final_geometry_convex         geometry
final_area_convex              float64
final_geometry_rectangular    geometry
final_area_rectangular         float64
dtype: object

In [ ]:
aa_rectaGeometry = aa["final_geometry_rectangular"]
aa_rectaGeometry.to_file(project_path + '/DATI/COMPLETE/Rettangoli_DegradoAreas/recta_gpkg.gpkg', driver='GPKG')
aa_othersElements.to_csv(project_path + '/DATI/COMPLETE/Rettangoli_DegradoAreas/recta_othersElements.csv')

In [ ]:
aa_rectaGeometry = gpd.read_file(project_path + '/DATI/COMPLETE/Rettangoli_DegradoAreas/recta_gpkg.gpkg')
aa_othersElements = pd.read_csv(project_path + '/DATI/COMPLETE/Rettangoli_DegradoAreas/recta_othersElements.csv')